Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
tf.nn.l2_loss?

In [25]:
batch_size = 128

hidden_layer_size = 1024

graph = tf.Graph()


with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    weights_hidden_layer = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_hidden_layer = tf.Variable(tf.zeros([num_labels]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
    
    logits = tf.matmul(hidden, weights_hidden_layer) + biases_hidden_layer
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    reqularizers = (tf.nn.l2_loss(weights) +  tf.nn.l2_loss(biases) + 
                    tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(biases_hidden_layer))
    
    loss += 8e-03 * reqularizers
    
    optimizer = tf.train.GradientDescentOptimizer(0.08).minimize(loss)
    
    
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights)) + biases, weights_hidden_layer) + biases_hidden_layer)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights)) + biases, weights_hidden_layer) + biases_hidden_layer)

                                   
                                 


In [28]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), ]
        batch_labels = train_labels[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if ((step % 500) == 0):
            print("Minibatch loss at step %i: %f" %(step, l))
            print('Minibatch accuracy: %.1f%%' % (accuracy(predictions, batch_labels)))
            print('Validation accuray: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels)))
    print('Testing accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels)))
        

Initialized
Minibatch loss at step 0: 2836.768066
Minibatch accuracy: 8.6%
Validation accuray: 21.8%
Minibatch loss at step 500: 1330.656982
Minibatch accuracy: 80.5%
Validation accuray: 79.6%
Minibatch loss at step 1000: 698.217529
Minibatch accuracy: 82.8%
Validation accuray: 81.0%
Minibatch loss at step 1500: 365.353088
Minibatch accuracy: 88.3%
Validation accuray: 83.0%
Minibatch loss at step 2000: 192.199615
Minibatch accuracy: 89.1%
Validation accuray: 83.8%
Minibatch loss at step 2500: 101.557137
Minibatch accuracy: 84.4%
Validation accuray: 83.4%
Minibatch loss at step 3000: 53.825668
Minibatch accuracy: 84.4%
Validation accuray: 86.0%
Testing accuracy: 92.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [48]:
batch_size = 1000

hidden_layer_size = 1024

graph = tf.Graph()


with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    weights_hidden_layer = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_hidden_layer = tf.Variable(tf.zeros([num_labels]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
    
    logits = tf.matmul(hidden, weights_hidden_layer) + biases_hidden_layer
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    reqularizers = (tf.nn.l2_loss(weights) +  tf.nn.l2_loss(biases) + 
                    tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(biases_hidden_layer))
    
    loss += 0
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights)) + biases, weights_hidden_layer) + biases_hidden_layer)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights)) + biases, weights_hidden_layer) + biases_hidden_layer)


num_steps = 100
                                   
                            
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    train_subset = train_dataset[0:1000,]
    train_labels_subset = train_labels[0:1000]
    for step in range(0, num_steps):
        feed_dict = {tf_train_dataset: train_subset, tf_train_labels: train_labels_subset}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if ((step % 25) == 1):
            print("Minibatch loss at step %i: %f" %(step, l))
            print('Minibatch accuracy: %.1f%%' % (accuracy(predictions, train_labels_subset)))
            print('Validation accuray: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels)))
    print('Testing accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels)))
        

Minibatch loss at step 1: 803.916626
Minibatch accuracy: 41.3%
Validation accuray: 34.3%
Minibatch loss at step 26: 0.000008
Minibatch accuracy: 100.0%
Validation accuray: 77.5%
Minibatch loss at step 51: 0.000003
Minibatch accuracy: 100.0%
Validation accuray: 77.5%
Minibatch loss at step 76: 0.000002
Minibatch accuracy: 100.0%
Validation accuray: 77.5%
Testing accuracy: 85.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [49]:
batch_size = 1000

hidden_layer_size = 1024

graph = tf.Graph()


with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    weights_hidden_layer = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_hidden_layer = tf.Variable(tf.zeros([num_labels]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
    
    weights_dropout = tf.nn.dropout(weights_hidden_layer, keep_prob = 0.5)
    
    logits = tf.matmul(hidden, weights_dropout) + biases_hidden_layer
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    reqularizers = (tf.nn.l2_loss(weights) +  tf.nn.l2_loss(biases) + 
                    tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(biases_hidden_layer))
    
    loss += 0
    

    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)


    
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights)) + biases, weights_hidden_layer) + biases_hidden_layer)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights)) + biases, weights_hidden_layer) + biases_hidden_layer)


num_steps = 100
                                   
                            
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    train_subset = train_dataset[0:1000,]
    train_labels_subset = train_labels[0:1000]
    for step in range(0, num_steps):
        feed_dict = {tf_train_dataset: train_subset, tf_train_labels: train_labels_subset}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if ((step % 25) == 1):
            print("Minibatch loss at step %i: %f" %(step, l))
            print('Minibatch accuracy: %.1f%%' % (accuracy(predictions, train_labels_subset)))
            print('Validation accuray: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels)))
    print('Testing accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels)))
        

Minibatch loss at step 1: 1207.234741
Minibatch accuracy: 35.0%
Validation accuray: 29.1%
Minibatch loss at step 26: 137.772491
Minibatch accuracy: 79.6%
Validation accuray: 74.5%
Minibatch loss at step 51: 6.727886
Minibatch accuracy: 94.6%
Validation accuray: 78.3%
Minibatch loss at step 76: 7.324823
Minibatch accuracy: 95.9%
Validation accuray: 78.7%
Testing accuracy: 85.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [86]:
batch_size = 128

hidden_layer_size1 = 1024
hidden_layer_size2 = 1024

graph = tf.Graph()


with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    ## Hidden layer one
    weights_hidden_layer1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size1]))
    biases_hidden_layer1 = tf.Variable(tf.zeros([hidden_layer_size1]))
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_layer1) + biases_hidden_layer1)

    ## Hidden layer two
    weights_hidden_layer2 = tf.Variable(tf.truncated_normal([hidden_layer_size1, hidden_layer_size2]))
    biases_hidden_layer2 = tf.Variable(tf.zeros([hidden_layer_size2]))
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights_hidden_layer2) + biases_hidden_layer2)
    
    ## Output layer
    weights = tf.Variable(tf.truncated_normal([hidden_layer_size2, num_labels]))
    weights_dropout = tf.nn.dropout(weights, keep_prob = 0.9)

    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden2, weights_dropout) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    reqularizers = (tf.nn.l2_loss(weights) +  tf.nn.l2_loss(biases) + 
                    tf.nn.l2_loss(weights_hidden_layer1) + tf.nn.l2_loss(biases_hidden_layer1) +
                    tf.nn.l2_loss(weights_hidden_layer2) + tf.nn.l2_loss(biases_hidden_layer2))
    
    loss += 8e-05 * reqularizers
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate=0.001, 
                                               global_step=global_step, 
                                              decay_steps=1,
                                              decay_rate=0.999)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    
    train_prediction = tf.nn.softmax(logits)
    
    test_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden_layer1) + biases_hidden_layer1)
    test_prediction = tf.nn.relu(tf.matmul(test_prediction, weights_hidden_layer2) + biases_hidden_layer2)
    test_prediction = tf.nn.relu(tf.matmul(test_prediction, weights) + biases)
    test_prediction = tf.nn.softmax(test_prediction)

    valid_prediction = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden_layer1) + biases_hidden_layer1)
    valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, weights_hidden_layer2) + biases_hidden_layer2)
    valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, weights) + biases)
    valid_prediction = tf.nn.softmax(valid_prediction)
                                   
                             

In [87]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), ]
        batch_labels = train_labels[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if ((step % 500) == 0):
            print("Minibatch loss at step %i: %f" %(step, l))
            print('Minibatch accuracy: %.1f%%' % (accuracy(predictions, batch_labels)))
            print('Validation accuray: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels)))
    print('Testing accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels)))
        

Initialized
Minibatch loss at step 0: 6824.564453
Minibatch accuracy: 5.5%
Validation accuray: 13.1%
Minibatch loss at step 500: 590.726440
Minibatch accuracy: 80.5%
Validation accuray: 78.9%
Minibatch loss at step 1000: 332.070007
Minibatch accuracy: 86.7%
Validation accuray: 80.5%
Minibatch loss at step 1500: 327.249634
Minibatch accuracy: 82.8%
Validation accuray: 81.3%
Minibatch loss at step 2000: 293.267700
Minibatch accuracy: 83.6%
Validation accuray: 81.8%
Minibatch loss at step 2500: 280.961914
Minibatch accuracy: 83.6%
Validation accuray: 81.9%
Minibatch loss at step 3000: 299.365265
Minibatch accuracy: 78.1%
Validation accuray: 82.5%
Testing accuracy: 89.9%
